In [ ]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

In [ ]:
!ls -la /content/

total 20
drwxr-xr-x 1 root root 4096 Mar  3 17:56 .
drwxr-xr-x 1 root root 4096 Mar  3 17:42 ..
drwxr-xr-x 4 root root 4096 Feb 28 14:26 .config
drwx------ 6 root root 4096 Mar  3 17:56 drive
drwxr-xr-x 1 root root 4096 Feb 28 14:27 sample_data


In [ ]:
from dotenv import load_dotenv
import lamini
import os
load_dotenv("/content/drive/MyDrive/DATA298/.env")
lamini.api_key = os.environ["LAMINI_API_KEY"]


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB         

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_personality_chain(retriever,personality,llm):
  instruction = f"You are a chatbot that communicates like {personality} during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like {personality} would. Use three sentences maximum and keep the answer concise."
  template = instruction + '''
Context: {context}
Question: {question}
Answer: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

In [ ]:
def parse_questions(file_path):
  with open(file_path) as fp:
    for line in fp.readlines():
      yield line.strip()

In [ ]:
def generate_answers(chain,questions_path):
  qa = []
  for question in parse_questions(questions_path):
    answer = chain.invoke(question)
    qa.append({"q":question,"a":answer})
  return qa


In [ ]:
def evaluate_personality_answers(qa,personality):
  template = '''
    You are a evaluator responsible for scoring the quality of the answers from the interview in terms of correctness and style.
    The following answers were generated by a chatbot pretending to be {personality}.
    For each question answer pair, produce only a score from 0 to 1 indicting the quality of the answer
    Question: {question}
    Answer: {answer}
    Score: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  llm = ChatOpenAI(model_name="gpt-4", temperature=0)

  evaluation_chain = (
      prompt
    | llm
    | StrOutputParser()
    )

  scores = []
  for d in qa:
    score = evaluation_chain.invoke({"personality": personality, "question": d["q"], "answer": d["a"]})
    scores.append(float(score))
  avg = sum(scores)/len(scores)
  return avg


# ***elon musk financial times 采访： https://www.ft.com/content/697d8d32-6ef9-4b4c-835a-3e9dcbdb431a  ***

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("elonmusk-ft.txt")

docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
elon_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = elon_db.as_retriever()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

elon_gpt35_chain = create_personality_chain(retriever,"elon musk",llm)



In [ ]:
qa = generate_answers(elon_gpt35_chain,"/content/elon_questions.txt")


In [ ]:
evaluate_personality_answers(qa,"elon musk")

0.9400000000000001

In [ ]:
qa

[{'q': 'What emerging technology do you believe will have the most significant impact on society in the next decade?',
  'a': "I believe artificial intelligence and neural interfaces will have the most significant impact on society in the next decade. These technologies have the potential to revolutionize how we interact with machines and each other, leading to a new era of human-machine symbiosis. It's crucial to ensure that these advancements are developed responsibly to benefit humanity."},
 {'q': "How do you envision the future of space exploration and humanity's role in it, especially considering SpaceX's missions?",
  'a': "SpaceX's mission is to make life multiplanetary, and I believe that is crucial for the future of humanity. We need to become a multiplanetary species to ensure the survival of our civilization. The future of space exploration is about expanding beyond Earth and creating a sustainable presence on other planets."},
 {'q': "What has been the most challenging deci

- add other personalities. find context and generate questions.




**- Go to the terminal to Run the ollama serve, the rest will work.**

In [ ]:
llm = ChatOllama(model="llama2", temperature=0)

elon_llama2_chain = create_personality_chain(retriever,"elon musk",llm)

In [ ]:
qa = generate_answers(elon_llama2_chain,"/content/elon_questions.txt")

In [ ]:
evaluate_personality_answers(qa,"elon musk")

0.685

# **sheldon**

# ***https://bigbangtrans.wordpress.com/series-1-episode-1-pilot-episode/***

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://bigbangtrans.wordpress.com/series-1-episode-1-pilot-episode/"
response = requests.get(url)
response

<Response [200]>

In [ ]:
parser = BeautifulSoup(response.content,"html.parser")
dialogue = parser.find("div",class_="entrytext").get_text()
with open("/content/drive/MyDrive/DATA298/sheldon/s1e1.txt","w") as fp:
  fp.write(dialogue)

In [ ]:
parser = BeautifulSoup(response.content,"html.parser")
current_episode = parser.find("li",class_="current_page_item")

siblings = current_episode.find_next_siblings()
urls = ["https://bigbangtrans.wordpress.com/series-1-episode-1-pilot-episode/"]
for episode in siblings:
  urls.append(episode.find("a")["href"])
urls

['https://bigbangtrans.wordpress.com/series-1-episode-1-pilot-episode/',
 'https://bigbangtrans.wordpress.com/series-1-episode-2-the-big-bran-hypothesis/',
 'https://bigbangtrans.wordpress.com/series-1-episode-3-the-fuzzy-boots-corollary/',
 'https://bigbangtrans.wordpress.com/series-1-episode-4-the-luminous-fish-effect/',
 'https://bigbangtrans.wordpress.com/series-1-episode-5-the-hamburger-postulate/',
 'https://bigbangtrans.wordpress.com/series-1-episode-6-the-middle-earth-paradigm/',
 'https://bigbangtrans.wordpress.com/series-1-episode-7-the-dumpling-paradox/',
 'https://bigbangtrans.wordpress.com/series-1-episode-8-the-grasshopper-experiment/',
 'https://bigbangtrans.wordpress.com/series-1-episode-9-the-cooper-hofstadter-polarization/',
 'https://bigbangtrans.wordpress.com/series-1-episode-10-the-loobenfeld-decay/',
 'https://bigbangtrans.wordpress.com/series-1-episode-11-the-pancake-batter-anomaly/',
 'https://bigbangtrans.wordpress.com/series-1-episode-12-the-jerusalem-duality/

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("/content/drive/MyDrive/DATA298/sheldon/s1e1.txt")

docs = loader.load()

In [ ]:
docs

[Document(page_content='Scene: A corridor at a sperm bank.\n\nSheldon: So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.\n\nLeonard: Agreed, what’s your point?\n\nSheldon: There’s no point, I just think it’s a good idea for a tee-shirt.\n\nLeonard: Excuse me?\n\nReceptionist: Hang on.\n\nLeonard: One across is Aegean, eight down is Nabakov, twenty-six across is MCM, fourteen down is… move your finger… phylum, which makes fourteen across Port-au-Prince. See, Papa Doc’s capital idea, that’s Port-au-Prince. Haiti.\n\nReceptionist: Can I help you?\n\nLeonard: Yes. Um, is this the High IQ sperm bank?\n\nReceptionist: If you have to ask, maybe you shouldn’t be here.\n\nSheldon: I think this is the place.\n\nReceptionist: Fill these out.\n\nLeonard: Thank-you. We’ll 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
sheldon_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = sheldon_db.as_retriever()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

sheldon_gpt35_chain = create_personality_chain(retriever,"sheldon cooper",llm)

In [ ]:
qa = generate_answers(sheldon_gpt35_chain,"/content/drive/MyDrive/DATA298/sheldon/sheldon-questions.txt")

In [ ]:
evaluate_personality_answers(qa,"sheldon cooper")

0.8499999999999999

In [ ]:
qa

[{'q': 'What would you consider the most significant flaw in the string theory framework?',
  'a': 'The lack of experimental evidence to support its predictions.'},
 {'q': 'If you had to choose one scientific principle to explain your life philosophy, what would it be and why?',
  'a': 'The Uncertainty Principle. Life is full of uncertainties and unpredictabilities, much like the behavior of particles at the quantum level. Embracing the unknown and accepting that we can never have complete control over every aspect of our lives is key to navigating the complexities of existence.'},
 {'q': 'In your opinion, which is more critical for advancing human knowledge: theoretical physics or experimental physics?',
  'a': 'Theoretical physics is more critical for advancing human knowledge.'},
 {'q': 'How would you redesign the periodic table to make it more efficient or informative?',
  'a': 'The periodic table is already a well-organized system based on atomic number and electron configuration.

# LINCOLN

# ***https://www.gutenberg.org/cache/epub/14004/pg14004.txt***

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("/content/drive/MyDrive/DATA298/lincoln/lincoln.txt")

docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
lincoln_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = lincoln_db.as_retriever()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

lincoln_gpt35_chain = create_personality_chain(retriever,"abraham lincoln",llm)

In [ ]:
qa = generate_answers(lincoln_gpt35_chain,"/content/drive/MyDrive/DATA298/lincoln/lincoln-questions.txt")

In [ ]:
evaluate_personality_answers(qa,"abraham lincoln")

0.945

In [ ]:
qa

[{'q': 'What motivated you to enter politics, and how did your early experiences shape your political views?',
  'a': 'I was motivated by my natural taste for politics and my desire to serve the people. My early experiences in the Illinois Legislature and as a stump-speaker shaped my political views by teaching me the importance of tact, ability, and readiness in debate. I entered politics to make a difference and serve my constituents to the best of my ability.'},
 {'q': 'How did your personal experiences with poverty and self-education influence your policies as President?',
  'a': "My poverty of books was the wealth of my life, shaping my character with earnestness, truthfulness, and purity. The few books I had, like Æsop's Fables and Pilgrim's Progress, were firmly fixed in my memory through repeated study. This mental training guided me in leading the nation through the great struggle with slavery."},
 {'q': 'What were the biggest challenges you faced in keeping the Union together

# Fine-tuning

In [ ]:
with open("/content/drive/MyDrive/DATA298/elon/elonmusk-ft.txt") as fp:
  interview = fp.read()


In [ ]:
import re

questions = re.findall(r"PC:.+\n",interview)
answers = re.findall(r"EM:.+\n",interview)

In [ ]:
len(questions)

30

In [ ]:
len(answers)

31

In [ ]:
questions[0]

'PC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n'

In [ ]:
answers[0]

'EM: I’m confident that we will be able to sell all the cars we can make. I mean, currently, the lead time for ordering a Tesla is ridiculously long, so our issue is not demand, it is production.\n'

In [ ]:
elon_data = []
for q,a in zip(questions,answers[1:]):
  elon_data.append({"input":q, "output":a})

In [ ]:
from lamini import LlamaV2Runner
llm = LlamaV2Runner()
llm.data = elon_data
llm.train()
llm.evaluate()



Your dataset id is: 917e569701e78db910c66f3cb9100252fcdddfd5bbaf59d49331e42690aa90d8 . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='917e569701e78db910c66f3cb9100252fcdddfd5bbaf59d49331e42690aa90d8')

Uploading data....
Upload to blob completed for data.
Data pairs uploaded to blob.
Training job submitted! Check status of job 5684 here: https://app.lamini.ai/train/5684
Cancelling job


Exception: Training failed